In [2]:
# ==========================
# Data Mining Midterm Project
# Brute Force, Apriori, FP-Growth
# ==========================

import pandas as pd
import itertools
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
import warnings
warnings.filterwarnings('ignore')

# ----- Utility Functions -----

def load_transactions(csv_path):
    """Load transactions from a CSV file and return a list of sets."""
    df = pd.read_csv(csv_path)
    if "Items" not in df.columns:
        df.columns = ["TID", "Items"]
    transactions = []
    for items in df["Items"]:
        parts = [i.strip() for i in items.split(",") if i.strip()]
        transactions.append(set(parts))
    return transactions


def one_hot_encode(transactions):
    """Convert list of transactions to one-hot encoded DataFrame."""
    all_items = sorted(set().union(*transactions))
    encoded = []
    for t in transactions:
        encoded.append({item: (item in t) for item in all_items})
    return pd.DataFrame(encoded, dtype=bool)


# ----- Brute Force Implementation -----

def enumerate_frequent_itemsets(transactions, minsup):
    """Brute-force enumeration of frequent itemsets."""
    items = sorted(set().union(*transactions))
    freq = {}
    k = 1
    while True:
        level_sets = [set(c) for c in itertools.combinations(items, k)]
        level_freq = {}
        for s in level_sets:
            support_val = sum(1 for t in transactions if s.issubset(t)) / len(transactions)
            if support_val >= minsup:
                level_freq[frozenset(s)] = support_val
        if not level_freq:
            break
        freq.update(level_freq)
        k += 1
    return freq


def generate_rules(freq_itemsets, minconf):
    """Generate association rules from frequent itemsets."""
    rules = []
    for itemset, suppL in freq_itemsets.items():
        s = set(itemset)
        if len(s) < 2:
            continue
        for i in range(1, len(s)):
            for X in itertools.combinations(s, i):
                X, Y = set(X), s - set(X)
                suppX = freq_itemsets.get(frozenset(X))
                if suppX:
                    conf = suppL / suppX
                    if conf >= minconf:
                        rules.append((X, Y, suppL, conf))
    return rules


# ----- Apriori & FP-Growth Wrappers -----

def run_apriori(onehot, minsup, minconf):
    """Run Apriori algorithm using mlxtend."""
    freq = apriori(onehot, min_support=minsup, use_colnames=True)
    rules = association_rules(freq, metric="confidence", min_threshold=minconf)
    return freq, rules


def run_fpgrowth(onehot, minsup, minconf):
    """Run FP-Growth algorithm using mlxtend."""
    freq = fpgrowth(onehot, min_support=minsup, use_colnames=True)
    rules = association_rules(freq, metric="confidence", min_threshold=minconf)
    return freq, rules


In [4]:
# --- Choose dataset and parameters ---
csv_path = "C:/Users/NANDE/Downloads/CS634_Midterm/data/amazon.csv"  # Change to any dataset file
minsup = 0.3
minconf = 0.6

# --- Load and preprocess ---
txs = load_transactions(csv_path)
onehot = one_hot_encode(txs)

print(" Dataset loaded successfully!")
print(f"Transactions: {len(txs)}")
print(f"One-hot encoded shape: {onehot.shape}")


 Dataset loaded successfully!
Transactions: 20
One-hot encoded shape: (20, 9)


In [14]:
# Step 3: Selecting Transactional Data
shop_company = {
    1: "amazon",
    2: "bestbuy",
    3: "kmart",
    4: "nike",
    5: "generic",
}

print("Welcome to Data Mining Midterm Project\n")
for num, name in shop_company.items():
    print(f"{num}. {name.title()}")

sel_comp = int(input("\nSelect a Shopping Company (1–5): "))
csv_path = f"../data/{shop_company[sel_comp]}.csv"

# Step 4: Providing Input for Minimum Support and Confidence
min_support = float(input("Enter Minimum Support (e.g., 0.3 for 30%): "))
confidence = float(input("Enter Minimum Confidence (e.g., 0.6 for 60%): "))

print(f"\nYou selected: {shop_company[sel_comp].title()}")
print(f"Support: {min_support}, Confidence: {confidence}")


Welcome to Data Mining Midterm Project

1. Amazon
2. Bestbuy
3. Kmart
4. Nike
5. Generic



Select a Shopping Company (1–5):  1
Enter Minimum Support (e.g., 0.3 for 30%):  0.4
Enter Minimum Confidence (e.g., 0.6 for 60%):  0.7



You selected: Amazon
Support: 0.4, Confidence: 0.7


In [15]:
print("=== Brute Force Algorithm ===")

freq_bf = enumerate_frequent_itemsets(txs, minsup)
rules_bf = generate_rules(freq_bf, minconf)

print(f"Frequent itemsets found: {len(freq_bf)}")
print(f"Association rules generated: {len(rules_bf)}")

if rules_bf:
    print("\n--- Rules (Brute Force) ---")
    for i, (X, Y, s, c) in enumerate(rules_bf, 1):
        print(f"Rule {i}: {X} -> {Y} | Support: {s:.2f}, Confidence: {c:.2f}")
else:
    print("No rules found at this threshold.")


=== Brute Force Algorithm ===
Frequent itemsets found: 15
Association rules generated: 20

--- Rules (Brute Force) ---
Rule 1: {'A Beginner’s Guide'} -> {'Java For Dummies'} | Support: 0.45, Confidence: 0.82
Rule 2: {'Java For Dummies'} -> {'A Beginner’s Guide'} | Support: 0.45, Confidence: 0.69
Rule 3: {'A Beginner’s Guide'} -> {'Java: The Complete Reference'} | Support: 0.45, Confidence: 0.82
Rule 4: {'Java: The Complete Reference'} -> {'A Beginner’s Guide'} | Support: 0.45, Confidence: 0.90
Rule 5: {'Head First Java 2nd Edition'} -> {'Android Programming: The Big Nerd Ranch'} | Support: 0.30, Confidence: 0.75
Rule 6: {'Android Programming: The Big Nerd Ranch'} -> {'Java For Dummies'} | Support: 0.45, Confidence: 0.69
Rule 7: {'Java For Dummies'} -> {'Android Programming: The Big Nerd Ranch'} | Support: 0.45, Confidence: 0.69
Rule 8: {'Java: The Complete Reference'} -> {'Android Programming: The Big Nerd Ranch'} | Support: 0.30, Confidence: 0.60
Rule 9: {'Java: The Complete Reference

In [16]:
print("\n=== Apriori Algorithm (mlxtend) ===")

af, ar = run_apriori(onehot, minsup, minconf)
print(f"Frequent itemsets found: {len(af)}")
print(f"Association rules generated: {len(ar)}")

if not ar.empty:
    print("\n--- Rules (Apriori) ---")
    for i, row in ar.head(10).iterrows():
        print(f"Rule {i+1}: {row['antecedents']} -> {row['consequents']} | Support: {row['support']:.2f}, Confidence: {row['confidence']:.2f}")
else:
    print("No rules found at this threshold.")



=== Apriori Algorithm (mlxtend) ===
Frequent itemsets found: 15
Association rules generated: 20

--- Rules (Apriori) ---
Rule 1: frozenset({'A Beginner’s Guide'}) -> frozenset({'Java For Dummies'}) | Support: 0.45, Confidence: 0.82
Rule 2: frozenset({'Java For Dummies'}) -> frozenset({'A Beginner’s Guide'}) | Support: 0.45, Confidence: 0.69
Rule 3: frozenset({'A Beginner’s Guide'}) -> frozenset({'Java: The Complete Reference'}) | Support: 0.45, Confidence: 0.82
Rule 4: frozenset({'Java: The Complete Reference'}) -> frozenset({'A Beginner’s Guide'}) | Support: 0.45, Confidence: 0.90
Rule 5: frozenset({'Head First Java 2nd Edition'}) -> frozenset({'Android Programming: The Big Nerd Ranch'}) | Support: 0.30, Confidence: 0.75
Rule 6: frozenset({'Android Programming: The Big Nerd Ranch'}) -> frozenset({'Java For Dummies'}) | Support: 0.45, Confidence: 0.69
Rule 7: frozenset({'Java For Dummies'}) -> frozenset({'Android Programming: The Big Nerd Ranch'}) | Support: 0.45, Confidence: 0.69
Rul

In [17]:
print("\n=== FP-Growth Algorithm (mlxtend) ===")

ff, fr = run_fpgrowth(onehot, minsup, minconf)
print(f"Frequent itemsets found: {len(ff)}")
print(f"Association rules generated: {len(fr)}")

if not fr.empty:
    print("\n--- Rules (FP-Growth) ---")
    for i, row in fr.head(10).iterrows():
        print(f"Rule {i+1}: {row['antecedents']} -> {row['consequents']} | Support: {row['support']:.2f}, Confidence: {row['confidence']:.2f}")
else:
    print("No rules found at this threshold.")



=== FP-Growth Algorithm (mlxtend) ===
Frequent itemsets found: 15
Association rules generated: 20

--- Rules (FP-Growth) ---
Rule 1: frozenset({'Android Programming: The Big Nerd Ranch'}) -> frozenset({'Java For Dummies'}) | Support: 0.45, Confidence: 0.69
Rule 2: frozenset({'Java For Dummies'}) -> frozenset({'Android Programming: The Big Nerd Ranch'}) | Support: 0.45, Confidence: 0.69
Rule 3: frozenset({'A Beginner’s Guide'}) -> frozenset({'Java For Dummies'}) | Support: 0.45, Confidence: 0.82
Rule 4: frozenset({'Java For Dummies'}) -> frozenset({'A Beginner’s Guide'}) | Support: 0.45, Confidence: 0.69
Rule 5: frozenset({'Java: The Complete Reference'}) -> frozenset({'Java For Dummies'}) | Support: 0.50, Confidence: 1.00
Rule 6: frozenset({'Java For Dummies'}) -> frozenset({'Java: The Complete Reference'}) | Support: 0.50, Confidence: 0.77
Rule 7: frozenset({'A Beginner’s Guide'}) -> frozenset({'Java: The Complete Reference'}) | Support: 0.45, Confidence: 0.82
Rule 8: frozenset({'Jav

In [23]:
print("\n=== FP-Growth Algorithm (mlxtend) ===")

ff, fr = run_fpgrowth(onehot, minsup, minconf)
print(f"Frequent itemsets found: {len(ff)}")
print(f"Association rules generated: {len(fr)}")

if not fr.empty:
    print("\n--- Rules (FP-Growth) ---")
    for i, row in fr.head(10).iterrows():
        print(f"Rule {i+1}: {row['antecedents']} -> {row['consequents']} | Support: {row['support']:.2f}, Confidence: {row['confidence']:.2f}")
else:
    print("No rules found at this threshold.")



=== FP-Growth Algorithm (mlxtend) ===
Frequent itemsets found: 15
Association rules generated: 20

--- Rules (FP-Growth) ---
Rule 1: frozenset({'Android Programming: The Big Nerd Ranch'}) -> frozenset({'Java For Dummies'}) | Support: 0.45, Confidence: 0.69
Rule 2: frozenset({'Java For Dummies'}) -> frozenset({'Android Programming: The Big Nerd Ranch'}) | Support: 0.45, Confidence: 0.69
Rule 3: frozenset({'A Beginner’s Guide'}) -> frozenset({'Java For Dummies'}) | Support: 0.45, Confidence: 0.82
Rule 4: frozenset({'Java For Dummies'}) -> frozenset({'A Beginner’s Guide'}) | Support: 0.45, Confidence: 0.69
Rule 5: frozenset({'Java: The Complete Reference'}) -> frozenset({'Java For Dummies'}) | Support: 0.50, Confidence: 1.00
Rule 6: frozenset({'Java For Dummies'}) -> frozenset({'Java: The Complete Reference'}) | Support: 0.50, Confidence: 0.77
Rule 7: frozenset({'A Beginner’s Guide'}) -> frozenset({'Java: The Complete Reference'}) | Support: 0.45, Confidence: 0.82
Rule 8: frozenset({'Jav

In [21]:
import time

start = time.time()
enumerate_frequent_itemsets(txs, minsup)
bf_time = time.time() - start

start = time.time()
run_apriori(onehot, minsup, minconf)
ap_time = time.time() - start

start = time.time()
run_fpgrowth(onehot, minsup, minconf)
fp_time = time.time() - start

print("\n=== Execution Time Comparison ===")
print(f"Brute Force: {bf_time:.4f} sec")
print(f"Apriori:     {ap_time:.4f} sec")
print(f"FP-Growth:   {fp_time:.4f} sec")



=== Execution Time Comparison ===
Brute Force: 0.0010 sec
Apriori:     0.0050 sec
FP-Growth:   0.0040 sec
